# Analyse des occupations et leur évolution dans le temps 

Dans ce carnet est proposée...

On met aussi en évidence l'évolution dans le temps ...

In [1]:
### Librairie à installer dans l'environnement conda
# qui sera choisi pour exécuter le carnet
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
### Librairies déjà installées avec Python
import pprint
import csv
import sys

import sqlite3 as sql

import time
import datetime
from dateutil import parser

from importlib import reload
from shutil import copyfile


In [4]:
### Importer un module de fonctions crées ad hoc
##  ATTENTION : le fichier 'sparql_functions.py' doit se trouver 
#   dans un dossier qui se situe dans le chemin ('path') de recherche
#   vu par le présent carnet Jupyter afin que
#   l'importation fonctionne correctement

# Add parent directory to the path
sys.path.insert(0, '..')

### If you want to add the parent-parent directory,
sys.path.insert(0, '../..')


import sparql_functions as spqf

## SPARQL Query qui récupère les données

On exécute la requête grâce à une fonction de la librairie locale qui réalise la mise en forme

In [5]:
## define SPARQL enpoint
endpoint = "https://query.wikidata.org/sparql"

In [6]:
query = """
SELECT DISTINCT ?item ?occupation ?occupationLabel
        WHERE {
            {
            {?item wdt:P106 wd:Q169470}
            UNION
            {?item wdt:P101 wd:Q413}  
          UNION 
          {?item wdt:P106 wd:Q11063}
            UNION
            {?item wdt:P101 wd:Q333} 
            UNION
              {?item wdt:P106 wd:Q155647}
            UNION
            {?item wdt:P101 wd:Q34362} 
              }
          
          ?item wdt:P31 wd:Q5;  # Any instance of a human.
              wdt:P569 ?birthDate;
                wdt:P106 ?occupation.
        BIND(REPLACE(str(?birthDate), "(.*)([0-9]{4})(.*)", "$2") AS ?year)
        FILTER(xsd:integer(?year) > 1350 )
          
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
        } 
     ORDER BY ?year
"""

In [7]:
### Executer la requête avec les fonctions de la librairie locale
qr = spqf.get_json_sparql_result(endpoint,query)

In [8]:
r = [l for l in spqf.sparql_result_to_list(qr)]
print(len(r))
r[:3]

69370


[['http://www.wikidata.org/entity/Q855257',
  'http://www.wikidata.org/entity/Q155647',
  'astrologer'],
 ['http://www.wikidata.org/entity/Q855257',
  'http://www.wikidata.org/entity/Q170790',
  'mathematician'],
 ['http://www.wikidata.org/entity/Q855257',
  'http://www.wikidata.org/entity/Q1622272',
  'university teacher']]

In [9]:
### Créer un DataFrame à partir du résultat
df_r = pd.DataFrame(r)
df_r.columns = ['personUri', 'occupationUri', 'occupationLabel']
df_r.head()

,personUri,occupationUri,occupationLabel
0,http://www.wikidata.org/entity/Q855257,http://www.wikidata.org/entity/Q155647,astrologer
1,http://www.wikidata.org/entity/Q855257,http://www.wikidata.org/entity/Q170790,mathematician
2,http://www.wikidata.org/entity/Q855257,http://www.wikidata.org/entity/Q1622272,university teacher
3,http://www.wikidata.org/entity/Q855257,http://www.wikidata.org/entity/Q4964182,philosopher
4,http://www.wikidata.org/entity/Q4121967,http://www.wikidata.org/entity/Q11063,astronomer


In [10]:
df_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69370 entries, 0 to 69369
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   personUri        69370 non-null  object
 1   occupationUri    69370 non-null  object
 2   occupationLabel  69370 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


## Créer une nouvelle table dans la base de données SQLite

In [11]:
### Se connecter à la base de données dans laquelle on va insérer
# le résultat de la requête SPARQL
cn = sql.connect('../../data/astronomers_import.db')
cn

In [14]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_r.to_sql(name='wdt_person_occupation', con=cn, if_exists='fail')
except Exception as e:
    print('Erreur: ',  e)

Erreur:  Table 'wdt_person_occupation' already exists.


In [15]:
### Vérifier que les données ont été importées correctement
cur = cn.cursor()
l = cur.execute("SELECT * FROM wdt_person_occupation limit 3").fetchall()

In [16]:
### On a mis le résultat de la requête SQL 
# dans une liste 'l' qu'on affiche avec une boucle 'for'
# dans le cadre d'une 'list comprehension'
a = [print(e) for e in l]

(0, 'http://www.wikidata.org/entity/Q855257', 'http://www.wikidata.org/entity/Q155647', 'astrologer')
(1, 'http://www.wikidata.org/entity/Q855257', 'http://www.wikidata.org/entity/Q170790', 'mathematician')
(2, 'http://www.wikidata.org/entity/Q855257', 'http://www.wikidata.org/entity/Q1622272', 'university teacher')


## Créer une table des occupations et une table de classement de domaines

La table qu'on vient de créer est une table de jointure entre les personnes et les occupations.
On va créer un table contenant les occupations dans la BD SQLite




```
```


In [28]:
cn

In [32]:
### Inspecter les données
q = """
SELECT DISTINCT occupationUri, occupationLabel, COUNT(*) as effectif
FROM wdt_person_occupation
GROUP BY occupationUri , occupationLabel 
ORDER BY effectif DESC;
"""


q = """
INSERT INTO wdt_occupation (wdt_uri,label)
SELECT occupationUri, occupationLabel
FROM wdt_person_occupation
GROUP BY occupationUri , occupationLabel 
HAVING COUNT(*) > 1 ;
"""

In [34]:
# cur = cn.cursor()
#r = cur.execute(q)
# cn.commit()